   # KACA算法
   ## 步骤：
   KACA(D,k):
   begin
   1. 由数据集D中生成初始等价类，等价类中各个元组在准标识符上值相等。
   2. 循环，知道不存在元组个数小于k的等价类
       1. 随机选择一个大小小于k的等价类C
       2. 计算C与其他所有等价类的距离
       3. 找到距C最近的等价类C'
       4. 将C和C'并为一类，并泛化C和C'
       循环结束
   3. 返回匿名表
   end;

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

In [2]:
AGECONFFILE = 'conf/age_hierarchy.txt'
WORKCONFFILE = 'conf/workclass_hierarchy.txt'
EDUCONFFILE = 'conf/edu_hierarchy.txt'
EDUNUMCONFFILE = 'conf/edunum_hierarchy.txt'
MARITALCONFFILE = 'conf/marital_hierarchy.txt'
OCCUCONFFILE = 'conf/occupation_hierarchy.txt'
RELATECONFFILE = 'conf/relate_hierarchy.txt'
RACECONFFILE = 'conf/race_hierarchy.txt'
SEXCONFFILE = 'conf/sex_hierarchy.txt'
COUNTRYCONFFILE = 'conf/country_hierarchy.txt'
ATTNAME = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
           'occupation', 'relationship', 'race', 'sex', 
            'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'incom']
towrite_sort = None

In [3]:
def readdata():
    df=pd.read_csv('adult.data.txt',names = ['age', 'workclass', 'fnlwgt','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','incom'],header=None)
    return df

In [4]:
class KAnonymity():
    def   __init__(self, records):
        self.records = records
        #self.confile = [AGECONFFILE, EDUCONFFILE, MARITALCONFFILE, RACECONFFILE]
        #self.confile = [AGECONFFILE, EDUCONFFILE, EDUNUMCONFFILE, MARITALCONFFILE,RACECONFFILE]
        self.confile = [AGECONFFILE, EDUCONFFILE]
    
    def anonymize(self, qi_names=['age', 'education', 'marital_status', 'race'], k=5):
        domains, gen_levels = {},{}
        qi_frequency ={}
        assert len(self.confile) == len(qi_names), 'number of config files  not equal to number of QI-names'
        generalize_tree = dict()
        
        for idx, name  in enumerate(qi_names):
            generalize_tree[name]=Tree(self.confile[idx])
            
        for qiname in qi_names:
            domains[qiname]=set()
            gen_levels[qiname]=0
        
        for idx,record in self.records.iterrows():
            qi_sequence = self._get_qi_values(record[:], qi_names, generalize_tree)
            #qi_sequence=tuple(record)
            #print(qi_sequence)
            if qi_sequence in qi_frequency:
                qi_frequency[qi_sequence].add(idx)
            else:
                qi_frequency[qi_sequence] = {idx}
                for j, value in enumerate(qi_sequence):
                    domains[qi_names[j]].add(value)
        
        # 迭代泛化
        while True:
            # 计算不满足k-匿名的记录数
            negcount = 0
            for qi_sequence, idxset in qi_frequency.items():
                if len(idxset) < k:
                    negcount += len(idxset)
            
            if negcount > k:
                # 如果>k，不满足k-匿名，继续泛化
                most_freq_att_num, most_freq_att_name = -1, None
                for qiname in qi_names:
                    if len(domains[qiname]) > most_freq_att_num:
                        most_freq_att_num = len(domains[qiname])
                        most_freq_att_name = qiname
                
                # 找出有最多不同值的属性
                generalize_att = most_freq_att_name
                qi_index = qi_names.index(generalize_att)
                domains[generalize_att] = set()
                
                # 往上泛化
                for qi_sequence in list(qi_frequency.keys()):
                    new_qi_sequence = list(qi_sequence)
                    new_qi_sequence[qi_index] =  generalize_tree[generalize_att].root[qi_sequence[qi_index]][0]
                    new_qi_sequence = tuple(new_qi_sequence)
                    
                    if new_qi_sequence == qi_sequence:
                        continue
                    if new_qi_sequence in qi_frequency:
                        qi_frequency[new_qi_sequence].update(
                            qi_frequency[qi_sequence])
                        qi_frequency.pop(qi_sequence, 0)
                    else:
                        qi_frequency[new_qi_sequence] = qi_frequency.pop(qi_sequence)
                    
                    #print(qi_frequency[new_qi_sequence])
                    
                    #if {0}.issubset(qi_frequency[new_qi_sequence]):
                        #print(new_qi_sequence)
                    domains[generalize_att].add(new_qi_sequence[qi_index])
                
                gen_levels[generalize_att] += 1
                
            
            else:
                genlvl_att = [0 for _ in range(len(qi_names))]
                dgh_att = [generalize_tree[name].level for name in qi_names]
                datasize = 0
                qiindex = [ATTNAME.index(name) for name in qi_names]

                # 确保输出文件与原始数据文件保持相同的顺序
                towriterecords = [[None,None,None,None,None,None,None,None,None,None,None,None,None,None,None]  
                                   for _ in range(len(self.records))]
                towriterecords = pd.DataFrame(towriterecords)
                global towrite_sort
                #towriterecords = None
                for qi_sequence, recordidxs in qi_frequency.items():
                    if len(recordidxs) < k:
                        continue
                    for idx in recordidxs:
                        record = self.records.loc[idx][:]
                        for i in range(len(qiindex)):
                            record[qiindex[i]] = qi_sequence[i]
                            genlvl_att[i] += generalize_tree[qi_names[i]].root[qi_sequence[i]][1]
                        record = list(map(str, record))
                        for i in range(len(record)):
                             if record[i] == '*' and i not in qiindex:
                                record[i] = '?'
                        tmp = pd.DataFrame(record).T
                        towriterecords.iloc[idx]=tmp.iloc[0]
                        if towrite_sort is None:
                            towrite_sort = tmp
                        else:
                            towrite_sort = towrite_sort.append(tmp,ignore_index=True)
                    datasize += len(recordidxs)
                    
                towriterecords.columns=['age', 'workclass', 'fnlwgt','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','incom']
                towrite_sort.columns=['age', 'workclass', 'fnlwgt','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','incom']
                towriterecords.to_csv('data/adult_%d_kanonymity.data' %k,index=False)
                print('qi names: ', qi_names)
                precision = self.calc_precision(genlvl_att, dgh_att, len(self.records), len(qi_names))
                distoration = self.calc_distoration([gen_levels[qi_names[i]] for i in range(len(qi_names))], dgh_att, len(qi_names))
                print('precision: {}, distoration: {}'.format(precision, distoration))
                break 
        return towriterecords
    
    def calc_precision(self, genlvl_att, dgh_att, datasize, attsize = 4):
        return 1 - sum([genlvl_att[i] / dgh_att[i] for i in range(attsize)])/(datasize*attsize)

    def calc_distoration(self, gen_levels_att, dgh_att, attsize):
        print('attribute gen level:', gen_levels_att)
        print('tree height:', dgh_att)
        return sum([gen_levels_att[i] / dgh_att[i] for i in range(attsize)]) / attsize

        
    def _get_qi_values(self, record, qi_names, generalize_tree):
        qi_index = [ATTNAME.index(name) for name in qi_names]
        seq = []
        for idx in qi_index:
            if idx == ATTNAME.index('age'):
                if record[idx] == -1:
                    seq.append('0-100')
                else:
                    seq.append(str(record[idx]))
            elif idx == ATTNAME.index('education_num'):
                seq.append(str(record[idx]))
            else:
                if record[idx] == '*':
                    record[idx] = generalize_tree[qi_names[idx]].highestgen
                seq.append(record[idx][1:])
        return tuple(seq)                    

In [5]:
class Tree:

    def __init__(self, confile):
        self.confile = confile
        self.root = dict()
        self.level = -1
        self.highestgen = ''
        self.buildTree()
        
    
    def buildTree(self):

        with open(self.confile, 'r') as rf:
            for line in rf:
                line = line.strip()
                if not line:
                    continue
                line = [col.strip() for col in line.split(',')]
                height = len(line)-1
                if self.level == -1:
                    self.level = height
                if not self.highestgen:
                    self.highestgen = line[-1]
                pre = None
                for idx, val in enumerate(line[::-1]):
                    self.root[val] = (pre, height-idx)
                    pre = val

In [6]:
df = readdata()
KAnony = KAnonymity(df)
newdf=KAnony.anonymize(qi_names=['age', 'education','education_num', 'marital_status', 'race'], k = 10)
newdf

qi names:  ['age', 'education', 'education_num', 'marital_status', 'race']
attribute gen level: [2, 2, 2, 2, 1]
tree height: [3, 4, 4, 3, 2]
precision: 0.43350736566239767, distoration: 0.5666666666666667


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,incom
0,0-50,State-gov,77516,ProfSchool,13-16,NonMarried,Adm-clerical,Not-in-family,Occident,Male,2174,0,40,United-States,<=50k
1,50-100,Self-emp-not-inc,83311,ProfSchool,13-16,Married,Exec-managerial,Husband,Occident,Male,0,0,13,United-States,<=50k
2,0-50,Private,215646,AdvancedSchool,6-12,Married,Handlers-cleaners,Not-in-family,Occident,Male,0,0,40,United-States,<=50k
3,50-100,Private,234721,AdvancedSchool,6-12,Married,Handlers-cleaners,Husband,Orient,Male,0,0,40,United-States,<=50k
4,0-50,Private,338409,ProfSchool,13-16,Married,Prof-specialty,Wife,Orient,Female,0,0,40,Cuba,<=50k
5,0-50,Private,284582,ProfSchool,13-16,Married,Exec-managerial,Wife,Occident,Female,0,0,40,United-States,<=50k
6,0-50,Private,160187,CompulsorySchool,1-5,Married,Other-service,Not-in-family,Orient,Female,0,0,16,Jamaica,<=50k
7,50-100,Self-emp-not-inc,209642,AdvancedSchool,6-12,Married,Exec-managerial,Husband,Occident,Male,0,0,45,United-States,>50k
8,0-50,Private,45781,ProfSchool,13-16,NonMarried,Prof-specialty,Not-in-family,Occident,Female,14084,0,50,United-States,>50k
9,0-50,Private,159449,ProfSchool,13-16,Married,Exec-managerial,Husband,Occident,Male,5178,0,40,United-States,>50k


In [7]:
towrite_sort

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,incom
0,0-50,State-gov,77516,ProfSchool,13-16,NonMarried,Adm-clerical,Not-in-family,Occident,Male,2174,0,40,United-States,<=50k
1,0-50,Private,94235,ProfSchool,13-16,NonMarried,Craft-repair,Unmarried,Occident,Male,0,0,40,United-States,<=50k
2,0-50,Private,195744,ProfSchool,13-16,NonMarried,Other-service,Not-in-family,Occident,Male,0,0,48,United-States,<=50k
3,0-50,Private,81896,ProfSchool,13-16,NonMarried,Prof-specialty,Not-in-family,Occident,Female,0,0,45,United-States,<=50k
4,0-50,Private,45781,ProfSchool,13-16,NonMarried,Prof-specialty,Not-in-family,Occident,Female,14084,0,50,United-States,>50k
5,0-50,Private,189265,ProfSchool,13-16,NonMarried,Exec-managerial,Not-in-family,Occident,Female,0,0,40,United-States,<=50k
6,0-50,Private,238329,ProfSchool,13-16,NonMarried,Sales,Not-in-family,Occident,Female,0,0,45,United-States,<=50k
7,0-50,Private,199915,ProfSchool,13-16,NonMarried,Adm-clerical,Own-child,Occident,Female,0,0,35,United-States,<=50k
8,0-50,Private,122272,ProfSchool,13-16,NonMarried,Adm-clerical,Own-child,Occident,Female,0,0,30,United-States,<=50k
9,0-50,Private,289748,ProfSchool,13-16,NonMarried,Exec-managerial,Not-in-family,Occident,Female,4650,0,48,United-States,<=50k


In [6]:
def cac_avg_age(df, type = 1,dellist = []):
    #df = readdata()
    newage=[]
    if type==1:
        for idx, record in df.iterrows():
            if record['age']=="0-25":
                newage.append(12.5)
            elif record['age']=="25-50":
                newage.append(37.5)
            elif record['age']=="50-75":
                newage.append(62.5)
            elif record['age']=="75-100":
                newage.append(87.5)
            elif record['age']=="0-50":
                newage.append(25)
            else :
                newage.append(75)
    else: 
        newage = df['age']
    
    if len(dellist)!=0:
        for i in dellist:
            del newage[i]
    return sum(newage)/len(newage)

In [7]:
#初始化参数
k = 10
real_avg_age = 0
dellist = []
mistake = {'real':[],'k_anonymize':[],'DP':[]}
xlist = []
ylist = []

records = readdata()
avg_age = cac_avg_age(records, type=2, dellist=dellist)
real_avg_age = avg_age
print("真实平均年龄：\n",real_avg_age)

k = int(input("请输入k值\n"))
KAnony = KAnonymity(records)
newdf=KAnony.anonymize(qi_names=['age', 'education'], k = k)
#newdf=KAnony.anonymize(k = k)
avg_age_k = cac_avg_age(newdf, type=1, dellist=dellist)

#差分攻击
mistake['k_anonymize'].append(avg_age_k)
print("K-匿名后平均年龄：",avg_age_k)

while(1):
    delp = int(input("请输入要删除的索引:(按-1结束)\n"))  
    if delp == -1:
        break
    dellist.append(delp)
    if len(dellist)!=0:
        for i in dellist:
            print("删除索引为{},年龄为{}".format(i,records['age'][i]))
            records = records.drop(i)

avg_age_k = cac_avg_age(newdf, type=1, dellist=dellist)
#差分攻击
mistake['k_anonymize'].append(avg_age_k)
print("执行以上操作后K-匿名后平均年龄：\n",avg_age_k)
age_predict = mistake['k_anonymize'][0]*(len(records)+len(dellist)) - avg_age_k*len(records)
print("根据前后数值差分攻击得到删除掉的年龄为：\n",age_predict)
del mistake['k_anonymize'][1]
if len(dellist)>0:
    del dellist[0]



真实平均年龄：
 38.58164675532078
请输入k值
5
qi names:  ['age', 'education']
attribute gen level: [1, 1]
tree height: [3, 4]
precision: 0.7083333333333334, distoration: 0.29166666666666663
K-匿名后平均年龄： 38.865130063572984
请输入要删除的索引:(按-1结束)
0
删除索引为0,年龄为39
请输入要删除的索引:(按-1结束)
-1
执行以上操作后K-匿名后平均年龄：
 38.86517199017199
根据前后数值差分攻击得到删除掉的年龄为：
 37.5
